In [1]:
import requests
import os.path # https://www.freecodecamp.org/news/how-to-check-if-a-file-exists-in-python/
import pyarrow as pa
import pyarrow.fs
import io

In [7]:
#Q8: how many live DataNodes are in the cluster?
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 25821052928 (24.05 GB)
Present Capacity: 15687532783 (14.61 GB)
DFS Remaining: 15429824512 (14.37 GB)
DFS Used: 257708271 (245.77 MB)
DFS Used%: 1.64%
Replicated Blocks:
	Under replicated blocks: 167
	Blocks with corrupt replicas: 0
	Missing blocks: 90
	Missing blocks (with replication factor 1): 90
	Low redundancy blocks with highest priority to recover: 167
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 172.19.0.5:9866 (project-4-pandas4-dn-1.project-4-pandas4_default)
Hostname: 7148d45c935e
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 257708271 (245.77 MB)
Non DFS Used: 10116742929 (9.42 GB)
DFS Remaining: 15429824512 (14.37 GB)
DFS Used%: 1.00%
DFS Re

In [8]:
import time
time.sleep(30)

In [9]:
#Q9: how are the blocks of single.csv distributed across the DataNode containers?
url = "http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS"
r = requests.get(url)
#r.raise_for_status()
file_status = r.json()

blockSize = file_status["FileStatus"]["blockSize"]
length = file_status["FileStatus"]["length"]
location_occurrences = {}
valid_blocks = []

idx = 0
while idx < length:
    url = f"http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={idx}&noredirect=true&length=100"
    r_1 = requests.get(url)
    status_code = r_1.status_code
    if status_code == 403:
        if not "lost" in location_occurrences:
           location_occurrences["lost"] = 0
        location_occurrences["lost"] += 1
    else:
        # url = f"http://boss:9870/webhdfs/v1/single.csv?op=OPEN&offset={idx}&noredirect=false&length=100"
        # r_2 = requests.get(url)
        valid_blocks.append(r_1.json())
        location = r_1.json()["Location"][7:19]
        if not location in location_occurrences:
            location_occurrences[location] = 0
        location_occurrences[location] += 1
    idx += blockSize
    
location_occurrences

{'lost': 90, '7148d45c935e': 77}

In [5]:
# valid_blocks

In [11]:
#Q10: how many times does the text "Single Family" appear in the remaining blocks of single.csv?
count = 0
hdfs = pa.fs.HadoopFileSystem("boss", 9000)

with hdfs.open_input_file("/single.csv") as f:
    for idx in range(len(valid_blocks)):
        offset = int(valid_blocks[idx]["Location"][102:])
        output = f.read_at(blockSize, offset) # output is bytes
        reader = (output.decode("UTF-8")) # byte to str
        count += reader.count("Single Family")

count

204798